In [1]:
import pandas as pd
import geopandas as gpd
import pathlib
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

import geojson
import h3

from geovoronoi import voronoi_regions_from_coords
crs4326 = {'init': 'epsg:4326'}

In [2]:
DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../../..' )

influences_df = pd.read_csv(DIR_ROOT + '/data/04_itinerary/influences.csv')
influences_df = influences_df[influences_df['domestic'] == True]

localities = pd.read_csv(DIR_ROOT + '/data/02_processed/localities.csv')
localities['geometry'] = localities['geometry'].apply(wkt.loads)
localities = gpd.GeoDataFrame(localities, crs=crs4326).set_geometry('geometry')

/home/casus-fwu036/anaconda3/envs/po2/lib/python3.9/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
bbox = localities.buffer(1).total_bounds
bounds = dict(geojson.Polygon([[[bbox[0], bbox[1]], [bbox[0], bbox[3]], [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]))
hex_ids = list(h3.polyfill(bounds, 4))

hex_df = gpd.GeoDataFrame(
    [{"geometry": Polygon(h3.h3_to_geo_boundary(x)), "id": x} for x in hex_ids], crs=crs4326
)
hex_df.set_index('id', inplace=True)

hex_df['id'] = hex_df.index
hex_df['geometry'] = hex_df.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x['id'])), axis=1)
hex_df

<ipython-input-3-7dfe25777e9d>:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox = localities.buffer(1).total_bounds


,geometry,id
id,,
847ab1dffffffff,"POLYGON ((4.67117 50.83127, 4.86913 50.89282, ...",847ab1dffffffff
8452067ffffffff,"POLYGON ((19.04710 42.42004, 18.83547 42.29510...",8452067ffffffff
845223bffffffff,"POLYGON ((21.37862 44.30889, 21.16912 44.18003...",845223bffffffff
8463825ffffffff,"POLYGON ((10.59403 51.30971, 10.40835 51.19220...",8463825ffffffff
84639adffffffff,"POLYGON ((8.13327 51.91435, 8.31925 51.97301, ...",84639adffffffff
...,...,...
846311dffffffff,"POLYGON ((15.42584 55.42638, 15.24331 55.30431...",846311dffffffff
8463323ffffffff,"POLYGON ((17.08202 54.73795, 16.89674 54.61353...",8463323ffffffff
847a927ffffffff,"POLYGON ((8.44068 51.22703, 8.62675 51.28682, ...",847a927ffffffff


In [4]:
def get_influence_origins(hex_geo):
    localities_in_hex = localities[localities.intersects(hex_geo)]

    infl_list = []
    for lihi, locality_in_hex in localities_in_hex.iterrows():
        inf_object_rows = influences_df[influences_df['name'] == locality_in_hex['name']]
        infl_list.extend(inf_object_rows['locality'].tolist())

    return { x: infl_list.count(x) for x in infl_list}

hex_df['infl'] = hex_df.apply(
    lambda x: get_influence_origins(x['geometry']), 
    axis=1
)
print(hex_df['infl'].tolist())

hex_df['all'] = hex_df.apply(
    lambda x: sum(x['infl'].values()), 
    axis=1
)

centers_to_check = ['Praha', 'Brno', 'Wien', 'Znojmo', 'Graz', 'Krems an der Donau', 'Linz']


for center in centers_to_check:
    hex_df[center] = hex_df.apply(
        lambda x: x['infl'][center] if center in x['infl'] else 0, 
        axis=1
    )

hex_df['others'] = hex_df.apply(
    lambda x: x['all'] - sum(x[center] for center in centers_to_check) , 
    axis=1
)



In [ ]:

hex_df.reset_index(drop=True).to_file(DIR_ROOT + '/data/06_outputs/hex_influence.geojson', driver="GeoJSON")

hex_df_geo = hex_df.copy()
hex_df_geo['centroid'] = hex_df_geo.apply(
    lambda x: x['geometry'].centroid, 
    axis=1
)
hex_df_geo = hex_df_geo.drop(columns=["geometry"])
hex_df_geo.set_geometry('centroid', inplace=True)
hex_df_geo = hex_df_geo.reset_index(drop=True)
hex_df_geo.to_file(DIR_ROOT + '/data/06_outputs/hex-centroids_influence.geojson', driver="GeoJSON")
